# Content based recommendation

Let us start by importing necessary libraries.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

Now let us load the pickle object as the data.